In [ ]:
import nltk
nltk.download('punkt')  # punkt номын санг татаж авах


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')  # punkt_tab номын санг татаж авах


In [ ]:
import requests
from collections import Counter
import re
from nltk.tokenize import word_tokenize
from transformers import pipeline
from flask import Flask, request, jsonify
import nltk

nltk.download('punkt')  # punkt номын санг татаж авах

# Текст оруулах
text = """Дэд бүтэц нээж, хувийн байгуулалгуудаа дэмжиж байж, улсын төсвийн хөрөнгө оруулалтаар хийж байж бизнесийн байгууллагуудын ажиллах орчин нөхцөлийг бүрдүүлэх ёстой. Ингэж байж зөвхөн уул уурхайгаас бус, бусад салбараас орлого олох боломжтой болно."""

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Текстийн өгөгдөл
texts = ["Эдийн засаг өсч байна", "Монголын соёл урлагийн талаар", "Шинжлэх ухааны шинэ ололтууд", "Улс төрийн асуудлууд"]
labels = [0, 1, 2, 3]  # Эдийн засаг, соёл урлаг, технологи, улс төр

# Токенжуулалт хийх
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, padding='post')

# Загвар бүтээх
model = models.Sequential([
    layers.Embedding(input_dim=1000, output_dim=64, input_length=len(padded_sequences[0])),
    layers.LSTM(64),
    layers.Dense(4, activation='softmax')
])

# Загварыг компил хийх
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Загварыг сургалтанд оруулах
model.fit(padded_sequences, np.array(labels), epochs=10)

# Шинээр текст ангилах
new_texts = ["Шинжлэх ухааны том ололт"]
new_sequences = tokenizer.texts_to_sequences(new_texts)
new_padded_sequences = pad_sequences(new_sequences, padding='post', maxlen=len(padded_sequences[0]))

# Текстийн ангилал авах
prediction = model.predict(new_padded_sequences)
predicted_label = np.argmax(prediction)
print(f"Текстийн ангилал: {predicted_label}")

# Алдаатай үгийг тодорхойлох
response = requests.post(
    "https://zuv.bichig.dev/api/check",
    json={"text": text}
)

marked_text = text  # Тэмдэглэсэн текстийг эхлээд эх текст гэж тодорхойлно.

if response.status_code == 200:
    corrections = response.json()
    corrected_words = []

    # Алдаатай үгийг доогуур зурах, санал болгох үг
    for mistake in corrections.get("data", []):
        wrong_word = mistake["word"]
        suggestions = mistake["suggestions"]
        corrected_words.append(wrong_word)

        # Алдаатай үгийн доогуур зурах
        marked_text = re.sub(
            rf'\b{wrong_word}\b',
            f'__{wrong_word}__',
            marked_text
        )
        print(f"Алдаатай үг: {wrong_word}, Санал болгож буй үгс: {', '.join(suggestions)}")
else:
    print(f"Серверийн алдаа: {response.status_code}")

print("\nТэмдэглэсэн текст:\n", marked_text)

# Үгний үндсийг задлах
words = word_tokenize(text)
stemmed_words = [re.sub(r"[^а-яА-Я]", "", word.lower()) for word in words if word.isalpha()]
word_count = Counter(stemmed_words)

# Хамгийн их давтагдсан 10 үг
most_common = word_count.most_common(10)
print("\nХамгийн их давтагдсан 10 үг:\n", most_common)

# Текст ангилагч (англи тексттэй ажилладаг тул орчуулга шаардагдана)
classifier = pipeline("zero-shot-classification")

# Ангиллын сонголтууд
labels = ["эдийн засаг", "спорт", "улс төр", "соёл урлаг", "технологи"]

# Бичвэрийг ангилах
result = classifier(text, candidate_labels=labels)
print("\nБичвэрийн агуулга:\n", result)

# Flask серверийн код
app = Flask(__name__)

@app.route('/check_text', methods=['POST'])
def check_text():
    data = request.get_json()
    text = data.get('text', '')
    response = requests.post("https://zuv.bichig.dev/api/check", json={"text": text})
    if response.status_code == 200:
        corrections = response.json()
        return jsonify(corrections)
    return {"error": "API алдаа гарлаа"}, 500

if __name__ == "__main__":
    app.run(debug=True)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.5000 - loss: 1.3814
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.7500 - loss: 1.3746
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 1.0000 - loss: 1.3677
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - accuracy: 1.0000 - loss: 1.3606
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 1.0000 - loss: 1.3533
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 1.0000 - loss: 1.3458
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step - accuracy: 1.0000 - loss: 1.3380
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 1.0000 - loss: 1.3299
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - accuracy: 1.0000 - loss: 1.3212
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 1.0000 - loss: 1.3121
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step


No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


Текстийн ангилал: 3
Серверийн алдаа: 405

Тэмдэглэсэн текст:
 Дэд бүтэц нээж, хувийн байгуулалгуудаа дэмжиж байж, улсын төсвийн хөрөнгө оруулалтаар хийж байж бизнесийн байгууллагуудын ажиллах орчин нөхцөлийг бүрдүүлэх ёстой. Ингэж байж зөвхөн уул уурхайгаас бус, бусад салбараас орлого олох боломжтой болно.

Хамгийн их давтагдсан 10 үг:
 [('байж', 3), ('дэд', 1), ('бтэц', 1), ('нээж', 1), ('хувийн', 1), ('байгуулалгуудаа', 1), ('дэмжиж', 1), ('улсын', 1), ('тсвийн', 1), ('хрнг', 1)]

Бичвэрийн агуулга:
 {'sequence': 'Дэд бүтэц нээж, хувийн байгуулалгуудаа дэмжиж байж, улсын төсвийн хөрөнгө оруулалтаар хийж байж бизнесийн байгууллагуудын ажиллах орчин нөхцөлийг бүрдүүлэх ёстой. Ингэж байж зөвхөн уул уурхайгаас бус, бусад салбараас орлого олох боломжтой болно.', 'labels': ['улс төр', 'соёл урлаг', 'эдийн засаг', 'спорт', 'технологи'], 'scores': [0.689415454864502, 0.146449014544487, 0.08029932528734207, 0.04334735870361328, 0.04048886150121689]}
 * Serving Flask app '__main__'
 * Debug mo

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
